# Grab Altmetric.com Data

## Request Functions
This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [1]:
from grab_util import *
import json
import pandas as pd
import numpy as np
import math


In [2]:
origin_folder = 'data/outputs/Nature'

## Grab altmetric.com Ids

In [3]:
def get_altmetric_id(doi):
    detail_id = ''
    res = grab_from_url_json('https://api.altmetric.com/v1/doi/' + doi)
    if res is not None:
        detail_id = res['altmetric_id']
    return detail_id

In [4]:
# all_articles_df = pd.read_excel(f'{origin_folder}/all.xlsx', usecols=['SO', 'DI'])
all_articles_df = pd.read_csv(f'{origin_folder}/all.csv', usecols=['SO', 'DI'])
df = pd.DataFrame(columns=['DI', 'altmetric_id'])

In [5]:
%%time
for index, doi in enumerate(all_articles_df['DI']):
    if doi in df['DI'].values:
        continue
    if index % 100 == 0:
        print(index)
    dfx = pd.DataFrame(columns=['DI', 'altmetric_id'])
    dfx['DI'] = [doi]
    dfx['altmetric_id'] = [get_altmetric_id(str(doi))]
    df = df.append(dfx, ignore_index=True)

df.to_csv(f'{origin_folder}/all_altmetric_id.csv', index=False)

nectionError: HTTPSConnectionPool(host='api.altmetric.com', port=443): Max retries exceeded with url: /v1/doi/10.1038/532150e (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11f482cf8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='api.altmetric.com', port=443): Max retries exceeded with url: /v1/doi/10.1038/532150e (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11f48e438>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='api.altmetric.com', port=443): Max retries exceeded with url: /v1/doi/10.1038/532150e (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11f48eb38>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectio

## Grab altmetric.com Details

In [6]:
def grab_detail_altmetric(doi, altmetric_id):
    df = pd.DataFrame()
    b_list = ['news outlets', 'blogs', 'policy', 'tweeters', 'weibo', 'facebook pages', 'wikipedia', 'redditors', 'f1000', 'video uploader', 'dimensions_citation', 'mendeley', 'citeulike']

    df['DI'] = [doi]
    df['altmetric_id'] = [altmetric_id]
    if altmetric_id != '' and not math.isnan(altmetric_id):
        altmetric_id = int(altmetric_id)
        news_anchor = 'news</dt><dd><a href="/details/' + str(altmetric_id) + '/news"><strong>'
        blogs_anchor = 'blogs</dt><dd><a href="/details/' + str(altmetric_id) + '/blogs"><strong>'
        policy_anchor = 'policy</dt><dd><a href="/details/' + str(altmetric_id) + '/policy-documents"><strong>'
        twitter_anchor = 'twitter</dt><dd><a href="/details/' + str(altmetric_id) + '/twitter"><strong>'
        weibo_anchor = 'weibo</dt><dd><a href="/details/' + str(altmetric_id) + '/weibo"><strong>'
        facebook_anchor = 'facebook</dt><dd><a href="/details/' + str(altmetric_id) + '/facebook"><strong>'
        wikipedia_anchor = 'wikipedia</dt><dd><a href="/details/' + str(altmetric_id) + '/wikipedia"><strong>'
        redditors_anchor = 'reddit</dt><dd><a href="/details/' + str(altmetric_id) + '/reddit"><strong>'
        f1000_anchor = 'f1000</dt><dd><a href="/details/' + str(altmetric_id) + '/f1000"><strong>'
        video_anchor = 'video</dt><dd><a href="/details/' + str(altmetric_id) + '/video"><strong>'
        dimensions_citation_anchor = 'dimensions_citation</dt><dd><a href="/details/' + str(altmetric_id) + '/citations"><strong>'
        mendeley_anchor = 'mendeley</dt><dd><a href="/details/' + str(altmetric_id) + '#mendeley-demographics"><strong>'
        citeulike_anchor = 'citeulike</dt><dd><strong>'
        c_list = [news_anchor, blogs_anchor, policy_anchor, twitter_anchor, weibo_anchor, facebook_anchor, wikipedia_anchor, redditors_anchor, f1000_anchor, video_anchor, dimensions_citation_anchor, mendeley_anchor, citeulike_anchor]

        end_anchor = '</strong>'

        res = grab_from_url_content('https://www.altmetric.com/details/' + str(altmetric_id))
        if res is not None:

            for i in range(0, len(c_list)):
                start_index = res.find(c_list[i])
                if start_index > 0:
                    start_index += len(c_list[i])
                    number = 0
                    end_index = res.find(end_anchor, start_index, start_index + 100)
                    number_temp = res[start_index: end_index]

                    if number_temp is not '':
                        number = number_temp
                    df[b_list[i]] = int(number)
                else:
                    df[b_list[i]] = 0
        else:
            for i in range(0, len(b_list)):
                df[b_list[i]] = 0
    else:
        for i in range(0, len(b_list)):
            df[b_list[i]] = 0

    return df

In [7]:
%%time
all_articles_df = pd.read_csv(f'{origin_folder}/all_altmetric_id.csv')
df = pd.DataFrame(columns=['DI'])
for index, row in all_articles_df.iterrows():
    if row['DI'] in df['DI'].values:
        continue
    if index % 100 == 0:
        print(index)
    dfx = grab_detail_altmetric(row['DI'], row['altmetric_id'])
    df = df.append(dfx, ignore_index=True)

df.to_csv(f'{origin_folder}/all_altmetric_detail.csv', index=False)

0
/Users/mikezhu/Dev/miniconda2/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
100
ConnectionError: HTTPSConnectionPool(host='www.altmetric.com', port=443): Read timed out.
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
ConnectionError: HTTPSConnectionPool(host='www.altmetric.com', port=443): Read timed out.
9200
9300
9400

## Grab altmetric.com's Tweets Detail

### Parser
We will get the html content from the url which is not listed as we want it be, so we need parser to parse them into listed data, in json form.

In [8]:
from html.parser import HTMLParser

class AltmetricHTMLParser(HTMLParser):
    tweets = []
    retweets = []
    articles = {'tweets': tweets, 'retweets': retweets}
    in_article = False
    is_reply = False
    has_article = False

    def handle_starttag(self, tag, attrs):
        if tag == 'body':
            self.has_article = False
        if tag == 'article':
            self.has_article = True
            self.in_article = True

        if self.in_article and (tag == 'a'):
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'reply'):
                    self.is_reply = True
                if self.is_reply and (attr[0] == 'href'):
                    self.tweets.append(attr[1].split('=')[1])
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_reply = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass

parser = AltmetricHTMLParser()

In [9]:
redirect_str = '<html><body>You are being <a href="https://www.altmetric.com/details/4236878">redirected</a>.</body></html>'

### Grab Data

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv(f'{origin_folder}/all_altmetric_id.csv', usecols=['DI', 'altmetric_id'], dtype={'altmetric_id':str})
df

,DI,altmetric_id
0,10.1038/d41586-017-00673-z,NaN
1,10.1038/nature.2016.21227,14940825
2,10.1038/nature.2016.21215,14945045
3,10.1038/nature.2016.21190,14941736
4,10.1038/nature.2016.21225,14914290
5,10.1038/nature.2016.21216,14913350
6,10.1038/nature.2016.21218,14891138
7,10.1038/nature.2016.21093,14885786
8,10.1038/d41586-017-00564-3,NaN
9,10.1038/nature.2016.21206,14858666


In [11]:
# %%time
# articles_dict = {}
# for index, row in df.iterrows():
#     if row['altmetric_id'] == '' or str(row['altmetric_id']).lower() == 'nan':
#         articles_dict[row['DI']] = {'altmetric_id': '', 'twitter_num': 0, 'tweets': []}
#         continue
#     # print(row['DI'], row['altmetric_id'])
#     grab_url = f'https://www.altmetric.com/details/{row["altmetric_id"]}/twitter/page:'
#     print(str(index), grab_url)
#     parser.articles = []
#     for i in range(1, 100000):
#         parser.feed(grab_from_url_content(grab_url + str(i), headers=headers))
#         print('page: ', str(i))
#         if not parser.has_article:
#             break
#     articles_dict[row['DI']] = {'altmetric_id': row['altmetric_id'], 'twitter_num': len(parser.articles), 'tweets': parser.articles}

In [12]:
# articles_dict = {}
def get_mention(source):
    headers = {
        'cookie': "_ga=GA1.2.501259425.1572141776; weibo_license_acknowledgement=false; explorer_user=WWs2eDNOOUVjVHJTcHRjSm9Oc0ZuZEh0eDRkTmlPd0tCWFMzODNyNlIweFBvdjllM3RkcmF0RVdtRWdvRDN4Ny0tQlNpeTBsWU5NK0ZNTnFpK1FpVFUzdz09--8bac413ca9fd623b9918b311729130cfbb4e66e1; _altmetric-explorer_session=eExKU3g1UmJhWXpXbmVtS05YZmxVS3YyaVNDSGdPL2JyaWVYaE9mRTdPT3RlYlRLaTdrRmRWcnZpeFMrMFJLb0FIVDBvNHI3N09MWS9HZDhDRDdIbTZPZElvMWNPNzMvTm5JVkovWThiQVplZVdTRlZheDBWYWdvd2lIMXo2NHJUZkFJd2MxcWpZSmpCaXVXRW9CYlZnZ1l4L1lXMGhIa1k1d0o1bllvS09uWDBQb3JXMWUrazZ2YWVwaUwzei93cDdUZkhwanFtSy9lSkxLUmdRdEIrZz09LS1DckIxZFJZUVBWNmhsTEJXcWtkR2x3PT0%3D--884ad24217914b545556a68678c2d803c65ce9cf; intercom-session-9dnltu6y=alcwSlZxZHMzZlY3Wm8yaFpvWFg1OGxEYVZ4MXRRUDMrbkk1Q01Bdm5XNXJkQXNZcVAyeEZYVHFYS3RUbWJSbi0tclFPUDlZcUVpMFZKQVlxdlMwRllVQT09--46bb4ebcf64063d6a3712dd82a1d2bff57eed87f, _ga=GA1.2.501259425.1572141776; weibo_license_acknowledgement=false; explorer_user=WWs2eDNOOUVjVHJTcHRjSm9Oc0ZuZEh0eDRkTmlPd0tCWFMzODNyNlIweFBvdjllM3RkcmF0RVdtRWdvRDN4Ny0tQlNpeTBsWU5NK0ZNTnFpK1FpVFUzdz09--8bac413ca9fd623b9918b311729130cfbb4e66e1; _altmetric-explorer_session=eExKU3g1UmJhWXpXbmVtS05YZmxVS3YyaVNDSGdPL2JyaWVYaE9mRTdPT3RlYlRLaTdrRmRWcnZpeFMrMFJLb0FIVDBvNHI3N09MWS9HZDhDRDdIbTZPZElvMWNPNzMvTm5JVkovWThiQVplZVdTRlZheDBWYWdvd2lIMXo2NHJUZkFJd2MxcWpZSmpCaXVXRW9CYlZnZ1l4L1lXMGhIa1k1d0o1bllvS09uWDBQb3JXMWUrazZ2YWVwaUwzei93cDdUZkhwanFtSy9lSkxLUmdRdEIrZz09LS1DckIxZFJZUVBWNmhsTEJXcWtkR2x3PT0%3D--884ad24217914b545556a68678c2d803c65ce9cf; intercom-session-9dnltu6y=alcwSlZxZHMzZlY3Wm8yaFpvWFg1OGxEYVZ4MXRRUDMrbkk1Q01Bdm5XNXJkQXNZcVAyeEZYVHFYS3RUbWJSbi0tclFPUDlZcUVpMFZKQVlxdlMwRllVQT09--46bb4ebcf64063d6a3712dd82a1d2bff57eed87f; Cookie_1=value",
        'User-Agent': "PostmanRuntime/7.19.0",
        'Accept': "*/*",
        'Cache-Control': "no-cache",
        'Postman-Token': "30d13c93-aa9a-44e1-84e0-8876f98ae1ba,b45c8eb6-7f77-4c48-b961-921c89280079",
        'Host': "www.altmetric.com",
        'Accept-Encoding': "gzip, deflate",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
    }
    
    for index, row in df.iterrows():
        if row['DI'] in articles_dict:
            continue
        if index % 100 == 0:
            print(str(index))
        if row['DI'] == '' or str(row['DI']).lower() == 'nan' or row['altmetric_id'] == '' or str(row['altmetric_id']).lower() == 'nan':
            articles_dict[row['DI']] = {'altmetric_id': '', f'{source}_num': 0, f'{source}s': []}
            continue

        articles = []
        grab_url = f'https://www.altmetric.com/explorer/json_data/mentions?identifier={row["DI"]}&mention_sources%5B%5D=type%3A{source}&scope=all&page='
        for i in range(1, 100000):
            articles_json = grab_from_url_json(grab_url + str(i), headers=headers)
            articles.extend(articles_json['data'])
            # print('page: ', str(i))
            if articles_json['lastPage']:
                break
            
        articles_dict[row['DI']] = {'altmetric_id': row['altmetric_id'], f'{source}_num': len(articles), f'{source}s': articles}
    return

In [ ]:
articles_dict = {}

In [13]:
%%time
get_mention('tweet')

or servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='www.altmetric.com', port=443): Max retries exceeded with url: /explorer/json_data/mentions?identifier=10.1038/540009a&mention_sources%5B%5D=type%3Atweet&scope=all&page=33 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x127d1fe48>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='www.altmetric.com', port=443): Max retries exceeded with url: /explorer/json_data/mentions?identifier=10.1038/540009a&mention_sources%5B%5D=type%3Atweet&scope=all&page=33 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x127d31668>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='www.altmetric.com', port=443): Max retries exceeded with url: /explorer/json_data/mentions?identifier

KeyboardInterrupt: 

In [17]:
with open(f'{origin_folder}/article_tweets_altmetric_x.json', "w+") as dump_f:
    dump_f.write(json.dumps(articles_dict))

In [ ]:
article_tweets = pd.read_json(f'{origin_folder}/article_tweets_altmetric_x.json', "r")

df_tweets = pd.DataFrame()

In [18]:
for doi in article_tweets:
    for tweet_chain in article_tweets[doi]['tweets']:
        for tweet in tweet_chain[1]:
            df_tweet = pd.DataFrame(columns=['doi', 'altmetric_id', 'tweet_id', 'tweet_url', 'postedAt', 'postType', 'originalPost'])
            df_tweet['doi'] = [doi]
            df_tweet['altmetric_id'] = [article_tweets[doi]['altmetric_id']]
            df_tweet['tweet_id'] = [str(tweet['url']).split('/')[-1]]
            df_tweet['tweet_url'] = [tweet['url']]
            df_tweet['postedAt'] = [tweet['postedAt']]
            df_tweet['postType'] = [tweet['postType']]
            df_tweet['originalPost'] = [str(tweet['originalPost'])]

            df_tweets = df_tweets.append(df_tweet, ignore_index=True)

In [19]:
df_tweets.to_csv(f'{origin_folder}/article_tweets_altmetric.csv', index=False)

In [ ]:
%%time
articles_dict = {}
get_mention('fbwall')

In [ ]:
with open(f'{origin_folder}/article_fbwalls_altmetric.json', "w+") as dump_f:
    dump_f.write(json.dumps(articles_dict))